
#  Anomaly detection in network traffic

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
plt.style.use("ggplot")
%matplotlib inline
import plotly
import plotly.graph_objs as go
import plotly.offline as py
from plotly.graph_objs import *
# plotly.offline.init_notebook_mode()
# plotly.tools.set_credentials_file(username='AARTURI', api_key='qiQqOxKJXDlziMFzaB8j')
py.init_notebook_mode(connected=True)
from datetime import datetime

## Features created from pcap (Wireshark)

In [42]:
cols = ["Number","Time","Source","Destination","Protocol","Length","Host","Destination Port","Source Port","Time_delta","Info","Bytes transfer","DateTime", "label"]

In [43]:

dfWireshark1 = pd.read_csv("dataset_wireshark" , sep=",", names=cols , index_col=None)
dfWireshark2 = pd.read_csv("dataset_wireshark2", sep=",", names=cols , index_col=None)
dfWireshark3 = pd.read_csv("dataset_wireshark3", sep=",", names=cols , index_col=None)
dfWireshark4 = pd.read_csv("dataset_selfgenerated", sep=",", names=cols , index_col=None)
dfWireshark = pd.concat([dfWireshark1, dfWireshark2, dfWireshark3, dfWireshark4], axis=0, join='inner').reset_index(drop=True)
dfWireshark = dfWireshark.drop("Number", axis = 1)
dfWireshark.to_csv("final_dataset")

/home/payas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning:

Columns (0,1,5,7,8,9,11) have mixed types. Specify dtype option on import or set low_memory=False.



## Preprocessing data

In [44]:
dfWireshark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100101 entries, 0 to 100100
Data columns (total 13 columns):
Time                100101 non-null object
Source              100101 non-null object
Destination         100101 non-null object
Protocol            100101 non-null object
Length              100101 non-null object
Host                229 non-null object
Destination Port    2533 non-null object
Source Port         2533 non-null object
Time_delta          100101 non-null object
Info                100101 non-null object
Bytes transfer      100101 non-null object
DateTime            100101 non-null object
label               0 non-null float64
dtypes: float64(1), object(12)
memory usage: 9.9+ MB


In [45]:
#Dense calculation, transforming date timestamps
dfWireshark['DateTime'] = pd.to_datetime(dfWireshark.DateTime,  errors='coerce')


In [46]:
dfWireshark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100101 entries, 0 to 100100
Data columns (total 13 columns):
Time                100101 non-null object
Source              100101 non-null object
Destination         100101 non-null object
Protocol            100101 non-null object
Length              100101 non-null object
Host                229 non-null object
Destination Port    2533 non-null object
Source Port         2533 non-null object
Time_delta          100101 non-null object
Info                100101 non-null object
Bytes transfer      100101 non-null object
DateTime            100098 non-null datetime64[ns]
label               0 non-null float64
dtypes: datetime64[ns](1), float64(1), object(11)
memory usage: 9.9+ MB


In [47]:
#Labeling data in two categories, Normal and Attack traffic
dfWireshark["label"] = "N"
#Attack traffic is hardcoding by previous analysis
dfWireshark.loc[3974:5124,'label'] = "A"
dfWireshark.loc[6745:7063,'label'] = "A"
dfWireshark.loc[8041:8359,'label'] = "A"


In [48]:
#verifying behavior
dfWireshark.loc[6745:7063]

,Time,Source,Destination,Protocol,Length,Host,Destination Port,Source Port,Time_delta,Info,Bytes transfer,DateTime,label
6745,18.028010,10.1.19.103,90.2.1.0,UDP,67,NaN,6892,63462,4.436555000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.898079,A
6746,18.028500,10.1.19.103,90.2.1.1,UDP,67,NaN,6892,63462,0.000490000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.898569,A
6747,18.028510,10.1.19.103,90.2.1.2,UDP,67,NaN,6892,63462,0.000010000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.898579,A
6748,18.028591,10.1.19.103,90.2.1.3,UDP,67,NaN,6892,63462,0.000081000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.898660,A
6749,18.028704,10.1.19.103,90.2.1.4,UDP,67,NaN,6892,63462,0.000113000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.898773,A
6750,18.028790,10.1.19.103,90.2.1.5,UDP,67,NaN,6892,63462,0.000086000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.898859,A
6751,18.028877,10.1.19.103,90.2.1.6,UDP,67,NaN,6892,63462,0.000087000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.898946,A
6752,18.028993,10.1.19.103,90.2.1.7,UDP,67,NaN,6892,63462,0.000116000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.899062,A
6753,18.029119,10.1.19.103,90.2.1.8,UDP,67,NaN,6892,63462,0.000126000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.899188,A
6754,18.029243,10.1.19.103,90.2.1.9,UDP,67,NaN,6892,63462,0.000124000,63462 > 6892 Len=25,67,2017-01-19 18:27:34.899312,A


In [49]:
#verifying behavior
dfWireshark.loc[0:3900]

,Time,Source,Destination,Protocol,Length,Host,Destination Port,Source Port,Time_delta,Info,Bytes transfer,DateTime,label
0,0,172.16.4.1,172.16.4.193,DHCP,348,NaN,68,67,0,DHCP ACK - Transaction ID 0x39132272,348,2017-01-27 19:53:08.210137,N
1,0.008974,172.16.4.193,224.0.0.252,LLMNR,69,NaN,5355,57650,0.008974,Standard query 0xf14f ANY Stewie-PC,69,2017-01-27 19:53:08.219111,N
2,0.107795,172.16.4.193,224.0.0.252,LLMNR,69,NaN,5355,57650,0.098821,Standard query 0xf14f ANY Stewie-PC,69,2017-01-27 19:53:08.317932,N
3,1.52807,172.16.4.193,224.0.0.252,LLMNR,66,NaN,5355,61613,1.42027,Standard query 0x8280 A isatap,66,2017-01-27 19:53:09.738207,N
4,1.63662,172.16.4.193,224.0.0.252,LLMNR,66,NaN,5355,61613,0.108548,Standard query 0x8280 A isatap,66,2017-01-27 19:53:09.846755,N
5,2.06019,172.16.4.193,172.16.4.255,NBNS,110,NaN,137,137,0.423573,Release NB STEWIE-PC<20>,110,2017-01-27 19:53:10.270328,N
6,2.06039,172.16.4.193,172.16.4.255,NBNS,110,NaN,137,137,0.000197,Release NB WORKGROUP<00>,110,2017-01-27 19:53:10.270525,N
7,2.0604,172.16.4.193,172.16.4.255,NBNS,110,NaN,137,137,8e-06,Release NB STEWIE-PC<00>,110,2017-01-27 19:53:10.270533,N
8,2.13618,172.16.4.193,172.16.4.255,NBNS,110,NaN,137,137,0.075783,Registration NB STEWIE-PC<00>,110,2017-01-27 19:53:10.346316,N
9,2.13638,172.16.4.193,172.16.4.255,NBNS,110,NaN,137,137,0.000198,Registration NB WORKGROUP<00>,110,2017-01-27 19:53:10.346514,N


In [50]:
#Rows to delete
dfWireshark.loc[(dfWireshark.label == "A") & (dfWireshark.Protocol != "UDP")]

,Time,Source,Destination,Protocol,Length,Host,Destination Port,Source Port,Time_delta,Info,Bytes transfer,DateTime,label


In [51]:
cols_groupby=["label"]
labels = pd.DataFrame({'count':dfWireshark.groupby("label").size()}).reset_index()
labels['count'].sum()

100101

### Creating dataframe with value information

In [52]:
dfMagicMike = dfWireshark.groupby(['Source',"Protocol","label", pd.Grouper(key='DateTime', freq='1s')]).agg({'Destination': pd.Series.nunique}).reset_index()

dfMagicMike.loc[dfMagicMike.Destination > 100]

,Source,Protocol,label,DateTime,Destination
19,10.1.19.102,UDP,A,2017-01-19 15:09:59,319
44,10.1.19.103,UDP,A,2017-01-19 18:27:34,319
689,172.16.4.193,UDP,A,2017-01-27 19:55:17,318
690,172.16.4.193,UDP,A,2017-01-27 19:55:18,256
692,172.16.4.193,UDP,A,2017-01-27 19:55:24,319
693,172.16.4.193,UDP,A,2017-01-27 19:55:25,256


In [53]:
#Correcting wrong labeling
#dfMagicMike = dfMagicMike.loc[((dfMagicMike.label == "A") & (dfMagicMike.Protocol== "UDP") & (dfMagicMike.Destination>1)) | (dfMagicMike.label == "N")]
dfMagicMike.loc[(dfMagicMike.label == "A")]

,Source,Protocol,label,DateTime,Destination
19,10.1.19.102,UDP,A,2017-01-19 15:09:59,319
44,10.1.19.103,UDP,A,2017-01-19 18:27:34,319
689,172.16.4.193,UDP,A,2017-01-27 19:55:17,318
690,172.16.4.193,UDP,A,2017-01-27 19:55:18,256
691,172.16.4.193,UDP,A,2017-01-27 19:55:19,1
692,172.16.4.193,UDP,A,2017-01-27 19:55:24,319
693,172.16.4.193,UDP,A,2017-01-27 19:55:25,256
694,172.16.4.193,UDP,A,2017-01-27 19:55:26,1


In [54]:
dfTrain = dfMagicMike.copy()

In [55]:
from sklearn.preprocessing import LabelEncoder

encs = dict()
dfEncoded = dfTrain.copy()  #.sample(frac=1)
for c in dfEncoded.columns:
    if dfEncoded[c].dtype == "object":
        encs[c] = LabelEncoder()
        dfEncoded[c] = encs[c].fit_transform(dfEncoded[c])

In [56]:
dfEncoded.groupby("label").describe()

Destination                                                            \
            count        mean         std  min     25%    50%    75%    max   
label                                                                         
0             8.0  223.625000  140.110097  1.0  192.25  287.0  319.0  319.0   
1          2005.0    1.521197    2.165363  1.0    1.00    1.0    1.0   36.0   

      Protocol             ...                Source                        \
         count       mean  ...     75%   max   count       mean        std   
label                      ...                                               
0          8.0  17.000000  ...    17.0  17.0     8.0  27.875000  16.898330   
1       2005.0  12.543142  ...    15.0  17.0  2005.0  50.842893  23.272506   

                                     
       min   25%   50%   75%    max  
label                                
0      0.0  28.0  37.0  37.0   37.0  
1      0.0  37.0  54.0  65.0  106.0  

[2 rows x 24 columns]

In [57]:
#Most important data training
dfEncoded.loc[dfEncoded.label == 0] # 0 = "Attack", 1 = "Normal"

,Source,Protocol,label,DateTime,Destination
19,0,17,0,2017-01-19 15:09:59,319
44,1,17,0,2017-01-19 18:27:34,319
689,37,17,0,2017-01-27 19:55:17,318
690,37,17,0,2017-01-27 19:55:18,256
691,37,17,0,2017-01-27 19:55:19,1
692,37,17,0,2017-01-27 19:55:24,319
693,37,17,0,2017-01-27 19:55:25,256
694,37,17,0,2017-01-27 19:55:26,1


## Training data

In [58]:
from sklearn.model_selection import train_test_split
dfEncoded2 = dfEncoded.copy()
train_cols = ["Protocol","Destination"]
y = dfEncoded.label
X = dfEncoded[train_cols]



In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=80)


### Isolation Forest

In [60]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score

iForest = IsolationForest(n_estimators=100, max_samples=256, 
                          contamination=0.0001)
iForest.fit(X_train)
prediction = iForest.predict(X_test)


acc = accuracy_score(prediction, y_test)
print ("Accuracy:", acc)

Accuracy: 0.9937965260545906


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/iforest.py:223: FutureWarning:

behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning:

threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.



In [61]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(prediction.shape)

(1207, 2)
(1207,)
(806, 2)
(806,)
(806,)


In [62]:
# Creating dataframe with Results
dfResults = X_test
dfResults["label"] = y_test
dfResults["prediction"] = prediction


/home/payas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/payas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [63]:
anomalies = dfResults.loc[dfResults.prediction==-1]
anomalies
#dfResults[(dfResults["prediction"]==-1) & (dfResults["label"]=="A")]


,Protocol,Destination,label,prediction
44,17,319,0,-1
689,17,318,0,-1
19,17,319,0,-1


In [64]:
dfResults.loc[(dfResults.label == 0)] # ONE miss prediction

,Protocol,Destination,label,prediction
693,17,256,0,1
44,17,319,0,-1
689,17,318,0,-1
19,17,319,0,-1
694,17,1,0,1


In [65]:
dfResults.groupby("prediction").describe()

Destination                                                    \
                 count        mean       std    min    25%    50%    75%   
prediction                                                                 
-1                 3.0  318.666667  0.577350  318.0  318.5  319.0  319.0   
 1               803.0    1.879203  9.292643    1.0    1.0    1.0    1.0   

                  Protocol            ...               label            \
              max    count       mean ...    75%   max  count      mean   
prediction                            ...                                 
-1          319.0      3.0  17.000000 ...   17.0  17.0    3.0  0.000000   
 1          256.0    803.0  12.676214 ...   15.0  17.0  803.0  0.997509   

                                               
                 std  min  25%  50%  75%  max  
prediction                                     
-1          0.000000  0.0  0.0  0.0  0.0  0.0  
 1          0.049875  0.0  1.0  1.0  1.0  1.0  

[2 rows x 24 columns]

In [66]:
dfResults.groupby("label").describe()

Destination                                                           \
            count        mean         std  min    25%    50%    75%    max   
label                                                                        
0             5.0  242.600000  137.758121  1.0  256.0  318.0  319.0  319.0   
1           801.0    1.563046    2.397151  1.0    1.0    1.0    1.0   36.0   

      Protocol            ...              prediction                      \
         count       mean ...    75%   max      count mean       std  min   
label                     ...                                               
0          5.0  17.000000 ...   17.0  17.0        5.0 -0.2  1.095445 -1.0   
1        801.0  12.665418 ...   15.0  16.0      801.0  1.0  0.000000  1.0   

                           
       25%  50%  75%  max  
label                      
0     -1.0 -1.0  1.0  1.0  
1      1.0  1.0  1.0  1.0  

[2 rows x 24 columns]

## Random forest

In [67]:
from sklearn.model_selection import train_test_split
dfEncoded2 = dfEncoded.copy()
train_cols = ["Protocol","Destination"]
y = dfEncoded.label
X = dfEncoded[train_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=80)


In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(max_depth=4, random_state=0, n_estimators = 20,verbose=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


In [69]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9987593052109182


In [70]:
dfResults2 = X_test
dfResults2["label"] = y_test
dfResults2["prediction"] = y_pred
dfResults2.loc[(dfResults2.label == 0)]


/home/payas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/payas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Protocol,Destination,label,prediction
693,17,256,0,0
44,17,319,0,0
689,17,318,0,0
19,17,319,0,0
694,17,1,0,1


# Graphics

In [71]:
dfplot = dfMagicMike.groupby(['Source','DateTime']).Destination.sum().reset_index()
dfplot['Source'] = dfplot['Source'] + '/' + dfplot['DateTime'].map(str)
data_plot = dfplot.sort_values(by='Destination',ascending = False, ).head(80)

In [72]:
data = [go.Bar(x=data_plot.Source,
            y=data_plot.Destination)
       ]

py.iplot(data, filename='jupyter-basic_bar')

In [121]:
 dfplot = dfMagicMike.loc[dfMagicMike.Destination > 1].sort_values(by='DateTime',ascending = True, )
sources = {}
data = []

for index, row in dfplot.iterrows():
    if row['Source'] not in sources:
        info = {}
        times = []
        dests = []
        info['time'] = times
        info['dest'] = dests
        sources[row['Source']] = info
        
    sources[row['Source']]['time'].append(row['DateTime'])
    sources[row['Source']]['dest'].append(row['Destination'])


for source in sources:
    
    trace = go.Scatter(
        x = sources[source]['time'],
        y = sources[source]['dest'],
        mode = 'lines',
        name = source
    )
    data.append(trace)

py.iplot(data, filename='line-mode')